In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from qiskit.circuit.library import CXGate
from rl_qoc.circuits import (
    get_circuit_context,
)
from rl_qoc import (
    QuantumEnvironment,
    ContextAwareQuantumEnvironment,
    RescaleAndClipAction,
)
from rl_qoc.environment import GateTarget
from rl_qoc.config import (
    QEnvConfig,
    BackendConfig,
    ExecutionConfig,
    RewardConfig,
)
import os

backend_config = BackendConfig(
    backend_name="statevector_simulator",
    n_qubits=2,
    initial_layout=[0, 1],
)
physical_qubits = [0, 1]

circuit_context = get_circuit_context(
    backend=backend_config.backend,
    physical_qubits=physical_qubits,
)

q_env_config = QEnvConfig(
    backend_config=backend_config,
    execution_config=ExecutionConfig(n_shots=1024),
    target=GateTarget(
        gate=CXGate(),
        physical_qubits=physical_qubits,
        circuit_context=circuit_context,
    ),
    reward=RewardConfig(method="state_fidelity"),
)

print("Circuit context")
circuit_context.draw("mpl")

In [2]:
from qiskit.visualization import plot_circuit_layout, plot_gate_map

if (
    q_env_config.backend_config.backend is not None
    and circuit_context.layout is not None
):
    fig = plot_circuit_layout(
        circuit_context, q_env_config.backend_config.backend, view="physical"
    )
    print(
        "Selected backend:",
        q_env_config.backend_config.backend.name,
        "backend version",
        q_env_config.backend_config.backend.version,
    )
elif (
    q_env_config.backend_config.backend is not None
    and q_env_config.backend_config.backend.coupling_map.size() != 0
):
    fig = plot_gate_map(q_env_config.backend_config.backend)
    print(
        "Selected backend:",
        q_env_config.backend_config.backend.name,
        "backend version",
        q_env_config.backend_config.backend.version,
    )
else:
    fig = None
    print("No backend selected, cannot plot circuit layout or gate map")

fig

No backend selected, cannot plot circuit layout or gate map


In [ ]:
use_context = False
if use_context:
    q_env = ContextAwareQuantumEnvironment(q_env_config)
else:
    q_env = QuantumEnvironment(q_env_config)
rescaled_env = RescaleAndClipAction(q_env, -1.0, 1.0)

In [5]:
from qiskit.visualization import plot_histogram
from qiskit_aer import AerSimulator
from qiskit.primitives import Sampler

# Create an AerSimulator instance
aer_sim = AerSimulator()

# Transpile the circuit for the simulator
t_qc = transpile(q_env.circuits[0], aer_sim)

# Run the simulation
# result = aer_sim.run(t_qc).result()
sampler = Sampler()
job = sampler.run(t_qc)
result = job.result()
counts = result.quasi_dists[0].binary_probabilities()

# Plot the histogram
plot_histogram(counts)